In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['posts']
posts_collection = db['posts']
posts_collection.drop()




In [11]:
from itertools import islice
import json
import pandas as pd
from datetime import datetime

with open ('data.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [ ]:
#1.1
import json
import random

from faker import Faker

# Create an instance of the Faker class
fake = Faker()

# Define the fields for posts, comments, and user information
post_fields = ["title", "content"]
comment_fields = ["text", "timestamp"]
user_fields = ["name", "email", "age"]

# Generate a list of 5 users with random information
users = []
for i in range(5):
    user = {}
    user["name"] = fake.name()
    user["email"] = fake.email()
    user["age"] = random.randint(18, 60)
    users.append(user)

# Generate random posts and comments for each user
data = []
for user in users:
    num_posts = random.randint(1, 100)
    for _ in range(num_posts):
        post = {}
        post["user"] = user["name"]
        post["user_email"] = user["email"]
        post["user_age"] = user["age"]
        post["post"] = {}
        post["post"]["title"] = f'Random title for {user["name"]}'
        post["post"]["content"] = f'Random content for {user["name"]} {fake.text()}'
        post["post"]["timestamp"] = fake.date_time().isoformat()
        num_comments = random.randint(0, 5)
        post["comments"] = []
        for _ in range(num_comments):
            comment = {}
            comment["user"] = random.choice(users)["name"]
            comment["user_email"] = random.choice(users)["email"]
            comment["user_age"] = random.choice(users)["age"]
            comment["comment"] = {}
            comment["comment"]["text"] = f"Random text {fake.text()}"
            comment["comment"]["timestamp"] = fake.date_time().isoformat()
            post["comments"].append(comment)
        data.append(post)

for item in data:
    posts_collection.insert_one(item)
    print(item)
    print('\n)')

    

In [ ]:
#1.2


import pandas as pd
# Buscar todos los usuarios mayores a 25

users = posts_collection.aggregate([
    {"$match": {"user_age": {"$gt": 25}}},
    {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])


# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df


In [ ]:
#1.3
# Buscar un usuario por direccion de correo

user = posts_collection.find_one({"user_email": "tmartin@example.org"})
# Imprimir el resultado
print(user)


In [5]:
#1.4
#Actualizar el contenido de la publicación de un usuario específico

user = posts_collection.update_one({"user": "Mrs. Christine Sawyer"}, {"$set": {"post.content": "Nuevo contenido para la publicación de Mrs. Christine Sawyer"}})
                                   
cambio = posts_collection.find({"user": "Mrs. Christine Sawyer"})

df = pd.DataFrame(list(cambio))
df

,_id,user,user_email,user_age,post,comments
0,65ed30b64fdb7c095fffc7b3,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Christopher Jenkins', 'user_email':..."
1,65ed30b64fdb7c095fffc7b4,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Amanda Johnson', 'user_email': 'wil..."
2,65ed30b64fdb7c095fffc7b5,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Lindsay Thornton', 'user_email': 't..."
3,65ed30b64fdb7c095fffc7b6,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Richard Bonilla', 'user_email': 'kr..."
4,65ed30b64fdb7c095fffc7b7,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Catherine Rios', 'user_email': 'kri..."


In [ ]:
#1.5
#Eliminar un documento de usuario y todos sus comentarios asociados.

user = posts_collection.delete_many({"user": "Lindsay Thornton"})

if user.deleted_count >= 1:
    print("El documento del usuario ha sido eliminado correctamente.")
else:
    print("El documento del usuario no pudo ser eliminado.")
    
comentarios = posts_collection.update_many({"comments.user": "Lindsay Thornton"}, {"$pull": {"comments":{"user": "Lindsay Thornton"}}})

if comentarios.modified_count > 0:
    print("Se eliminaron", comentarios.modified_count, "comentarios de Lindsay Thornton correctamente.")
else:
    print("No se encontraron comentarios de Lindsay Thornton o no se pudieron eliminar.")

In [ ]:
#2.1
# Buscar todos los usuarios cuyo nombre de usuario contenga la palabra S̈mith.̈ (Expresiones regulares)
users = posts_collection.aggregate([
    {"$match": {"user": {"$regex": "Smith", "$options": "i"}}},
    {"$group": {"_id": "$user"}},
    {"$project": {"_id": 0, "user": "$_id"}}
])


# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df


In [ ]:
#2.2
# Buscar publicaciones creadas después de una fecha específica (por ejemplo, 2000-01-01)


date = datetime(2000, 1, 1)


posts = posts_collection.find(
    {"post.timestamp": {"$gt": date}},
    {"user": 1, "post.timestamp": 1, "_id": 0} # Seleccionar solo el nombre del usuario y la fecha
)


df = pd.DataFrame(list(posts))
df.rename(columns={"post": "timestamp"}, inplace=True)  # Renombrar la columna "post" a "timestamp"
df


In [51]:
#2.3 Buscar usuarios que aún no han publicado ningún comentario

import pandas as pd

users = posts_collection.aggregate([
    {"$unwind": "$comments"},
    {"$group": {"_id": "$user", "comments": {"$sum": 1}}},
    {"$match": {"comments": 0}},
    {"$project": {"_id": 0, "user": "$_id"}}
])

# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df

""


In [ ]:
#2.4
# Encontrar la edad promedio de todos los usuarios.

users = posts_collection.aggregate([
{"$group":{"_id": None,"edad_promedio": {"$avg": "$user_age"}}}]
)

df = pd.DataFrame(list(users))
df

In [ ]:
#2.5
#Buscar documentos donde la longitud del contenido de la publicación sea superior a 100 caracteres
documentos = posts_collection.find({"$expr": {"$gt":[{"$strLenCP":"$post.content"}, 100]}})

df = pd.DataFrame(list(documentos))
df


In [ ]:
#3.1
#Buscar usuarios que tengan más de 20 años pero menos de 30. (Combinar operadores)

users = posts_collection.aggregate([
{"$match": {"user_age": {"$gt": 25, "$lt": 30}}},
{"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
{"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

In [ ]:
#3.2

# Recuperar solo el nombre de usuario y la edad del usuario de todos los documentos. (Proyección
users = posts_collection.find({}, {"user": 1, "user_age": 1, "_id": 0})
df = pd.DataFrame(list(users))
df

In [ ]:
#3.3 Buscar todas las publicaciones y excluir el campo ”comentarios”.

posts = posts_collection.find({}, {"_id": 0,"comments": 0})
df = pd.DataFrame(list(posts))
df


In [8]:
#3.4
# Buscar usuarios con al menos un comentario en su matriz ”comentarios”
userConComentarios = posts_collection.find({"comments": {"$exists": True, "$not": {"$size": 0}}})

df = pd.DataFrame(list(userConComentarios))
df

,_id,user,user_email,user_age,post,comments
0,65ed30b64fdb7c095fffc7a4,Lindsay Thornton,alexanderwheeler@example.org,28,"{'title': 'Random title for Lindsay Thornton',...","[{'user': 'Kristi Cooley', 'user_email': 'clay..."
1,65ed30b64fdb7c095fffc7a5,Lindsay Thornton,alexanderwheeler@example.org,28,"{'title': 'Random title for Lindsay Thornton',...","[{'user': 'Beth Mendez', 'user_email': 'carrie..."
2,65ed30b64fdb7c095fffc7a7,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Joshua Murray', 'user_email': 'tmar..."
3,65ed30b64fdb7c095fffc7a8,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Erin Smith', 'user_email': 'victori..."
4,65ed30b64fdb7c095fffc7a9,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Lucas Rogers', 'user_email': 'kmorr..."
...,...,...,...,...,...,...
1212,65ed30b74fdb7c095fffcd39,Travis Nicholson,tommy81@example.org,42,"{'title': 'Random title for Travis Nicholson',...","[{'user': 'Beth Mendez', 'user_email': 'thomps..."
1213,65ed30b74fdb7c095fffcd3b,Travis Nicholson,tommy81@example.org,42,"{'title': 'Random title for Travis Nicholson',...","[{'user': 'Zachary King', 'user_email': 'edwar..."
1214,65ed30b74fdb7c095fffcd3c,Travis Nicholson,tommy81@example.org,42,"{'title': 'Random title for Travis Nicholson',...","[{'user': 'Robert Wood', 'user_email': 'stephe..."
1215,65ed30b74fdb7c095fffcd3d,Travis Nicholson,tommy81@example.org,42,"{'title': 'Random title for Travis Nicholson',...","[{'user': 'Richard Bonilla', 'user_email': 'ca..."


In [ ]:
#3.5 Buscar comentarios publicados por usuarios mayores de 40 años. (Consultas anidadas)

users = posts_collection.aggregate([
    {"$unwind": "$comments"},
    {"$match": {"user_age": {"$gt": 40}}},
    {"$project": {"_id": 0, "user": "$comments.user", "user_age": 1, "comment": "$comments.comment"}}
])

df = pd.DataFrame(list(users))
df

In [ ]:
#4.1
# Ordenar todos los usuarios por edad en orden descendente

users = posts_collection.aggregate([
{"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
{"$sort": {"user_age": -1}},
{"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

In [25]:
#4.2

#Buscar los top 3 usuarios principales con las publicaciones más recientes (en función de la marca de tiempo).

users = posts_collection.aggregate([
    {"$sort": {"post.timestamp": -1}},
    {"$group": {"_id": "$user", "latest_post": {"$first": "$post.timestamp"}}},
    {"$sort": {"latest_post": -1}},
    {"$limit": 3},
    {"$project": {"_id": 0, "user": "$_id", "latest_post": 1}}
])

df = pd.DataFrame(list(users))
df

,latest_post,user
0,2024-02-28 20:30:48.385,Michael Smith
1,2024-02-26 09:53:04.022,Erin Smith
2,2024-02-04 15:56:25.075,Gary Best


In [8]:
#4.3 Omitir los primeros 5 resultados y recuperar los próximos 10 usuarios.

users = posts_collection.aggregate([
    # {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$skip": 5},
    {"$limit": 10},
    # {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

,_id,user,user_email,user_age,post,comments
0,65ed2973c10f236f84942f3e,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Lucas Rogers', 'user_email': 'kmorr..."
1,65ed2973c10f236f84942f3f,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Debra Smith DDS', 'user_email': 'sa..."
2,65ed2973c10f236f84942f40,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...",[]
3,65ed2973c10f236f84942f41,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Jacob Carroll', 'user_email': 'tmar..."
4,65ed2973c10f236f84942f42,Will Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...",[]
5,65ed2973c10f236f84942f43,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Jeffrey Perez', 'user_email': 'carr..."
6,65ed2973c10f236f84942f44,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Mrs. Christine Sawyer', 'user_email..."
7,65ed2973c10f236f84942f45,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Michael Smith', 'user_email': 'kris..."
8,65ed2973c10f236f84942f46,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...",[]
9,65ed2973c10f236f84942f47,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Joshua Murray', 'user_email': 'rene..."


In [ ]:
#5.1
#1. Utilizar el pipeline de agregación para calcular el número total de usuarios.

users = posts_collection.aggregate([
{"$group": {"_id": "$user"}},
{"$group": {"_id": None, "Cantidad": {"$sum": 1}}},
{"$project": {"_id": 0, "Cantidad": 1}}
])

df = pd.DataFrame(list(users))
df


In [25]:
#5.2

#  Encontrar al usuario con el contenido de publicación más largo (en función del número de caracteres)

users = posts_collection.aggregate([
    {"$project": {"user": 1, "largo_Contenido": {"$strLenCP": "$post.content"}}},
    {"$sort": {"largo_Contenido": -1}},
     {"$limit": 1}
])

df = pd.DataFrame(list(users))
df

,_id,user,largo_Contenido
0,65ed2bf0c10f236f849439cf,Christopher Jenkins,238


In [35]:
#5.3  Agrupar a los usuarios por edad y contar la cantidad de usuarios en cada grupo de edad.

users = posts_collection.aggregate([
    {"$group": {"_id": {"user": "$user", "age": "$user_age"}}},  
    {"$group": {"_id": "$_id.age", "Cantidad": {"$sum": 1}}},
    {"$project": {"_id": 0, "Edad": "$_id", "Cantidad": 1}},
    {"$sort": {"Edad": 1}}
])


df = pd.DataFrame(list(users))
df


,Cantidad,Edad
0,2,18
1,2,19
2,1,21
3,1,23
4,1,24
5,1,27
6,2,28
7,1,29
8,1,30
9,1,32


In [9]:
#5.4
#Calcular la longitud promedio de los comentarios en todos los comentarios.
cantComentarios= posts_collection.aggregate([
    {
        "$unwind": "$comments"  # Desenrolla la matriz de comentarios
    },
    {
        "$group": {
            "_id": None,
            "longitud_promedio_comentarios": {"$avg": {"$strLenCP": "$comments.comment.text"}}
        }
    }
])

df = pd.DataFrame(list(cantComentarios))
df

,_id,longitud_promedio_comentarios
0,None,159.292291


In [36]:
#5.5 Cambiar la forma como se presentan los datos, de tal forma que cada usuario tenga la lista de todos
# los posts asociados a él. No se deben agregar el detalle de los comentarios, solo un nuevo campo con
# el total de comentarios

users = posts_collection.aggregate([
    {"$unwind": "$comments"},
    {"$group": {"_id": "$user", "posts": {"$push": "$post"}, "total_comments": {"$sum": 1}}},
    {"$project": {"_id": 0, "user": "$_id", "posts": 1, "total_comments": 1}}
])

df = pd.DataFrame(list(users))
df

,posts,total_comments,user
0,"[{'title': 'Random title for Erin Smith', 'con...",143,Erin Smith
1,"[{'title': 'Random title for Kristi Cooley', '...",212,Kristi Cooley
2,"[{'title': 'Random title for Debra Smith DDS',...",7,Debra Smith DDS
3,"[{'title': 'Random title for Zachary King', 'c...",22,Zachary King
4,"[{'title': 'Random title for Jacob Carroll', '...",237,Jacob Carroll
5,"[{'title': 'Random title for Margaret Flores',...",19,Margaret Flores
6,[{'title': 'Random title for Christopher Jenki...,196,Christopher Jenkins
7,"[{'title': 'Random title for Amanda Johnson', ...",207,Amanda Johnson
8,"[{'title': 'Random title for Jeffrey Perez', '...",229,Jeffrey Perez
9,"[{'title': 'Random title for Beth Mendez', 'co...",7,Beth Mendez


In [ ]:
#6.1

# Incrementar la edad de todos los usuarios en 1 con operadores

posts_collection.update_many({}, {"$inc": {"user_age": 1}})

users = posts_collection.aggregate([
    {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])


df = pd.DataFrame(list(users))
df

In [50]:
#6.2

# Establecer el campo ”contenido” de una publicación específica en una cadena vacía.

posts_collection.update_one({"post.title": "Random title for Mrs. Christine Sawyer"}, {"$set": {"post.content": ""}})

post = posts_collection.find({"post.content": ""})

df = pd.DataFrame(list(post))
df

,_id,user,user_email,user_age,post,comments
0,65ed0d6aa4fe388ae296fce5,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Joshua Murray', 'user_email': 'tmar..."
1,65ed0d6aa4fe388ae296fcf1,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Christopher Jenkins', 'user_email':..."
2,65ed0d6aa4fe388ae296fcf6,Kayla Hill,snyderkathy@example.net,29,"{'title': 'Random title for Kayla Hill', 'cont...","[{'user': 'Jon Rodriguez', 'user_email': 'jose..."


In [38]:
#6.3 . Eliminar todos los comentarios de la publicación de un usuario.

posts_collection.update_one({"user": "Mrs. Christine Sawyer"}, {"$unset": {"comments": ""}})
post = posts_collection.find({"user": "Mrs. Christine Sawyer"})
df = pd.DataFrame(list(post))
df


,_id,user,user_email,user_age,post,comments
0,65ed2bedc10f236f84943610,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,NaN
1,65ed2bedc10f236f84943611,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Amanda Johnson', 'user_email': 'wil..."
2,65ed2bedc10f236f84943612,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Lindsay Thornton', 'user_email': 't..."
3,65ed2bedc10f236f84943613,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Richard Bonilla', 'user_email': 'kr..."
4,65ed2bedc10f236f84943614,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Catherine Rios', 'user_email': 'kri..."


In [12]:
#6.4
#Eliminar todos los usuarios con direcciones de correo electrónico que terminen en ”@example.org

borrarUsuarios = posts_collection.delete_many({"user_email": {"$regex": "@example.org$"}})

print("Se eliminaron", borrarUsuarios.deleted_count, "usuarios.")


Se eliminaron 660 usuarios.


In [ ]:
#7.1

# Crear un índice en el campo ”user_email” para búsquedas de correo electrónico más rápidas.
posts_collection.create_index([("user_email", 1)])

df = pd.DataFrame(list(posts_collection.index_information()))
df


In [53]:
#7.2

# Explicar el plan de consulta para encontrar usuarios con edad mayor a 30.


posts_collection.find({"user_age": {"$gt": 30}}).explain()







{'explainVersion': '1',
 'queryPlanner': {'namespace': 'posts.posts',
  'indexFilterSet': False,
  'parsedQuery': {'user_age': {'$gt': 30}},
  'queryHash': 'A95D609D',
  'planCacheKey': 'A95D609D',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'user_age': {'$gt': 30}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 860,
  'executionTimeMillis': 1,
  'totalKeysExamined': 0,
  'totalDocsExamined': 1435,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'user_age': {'$gt': 30}},
   'nReturned': 860,
   'executionTimeMillisEstimate': 0,
   'works': 1436,
   'advanced': 860,
   'needTime': 575,
   'needYield': 0,
   'saveState': 1,
   'restoreState': 1,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 1435},
  'allPlansExecution': []},
 'command': {'find': 'posts',
  'filter': {'use

In [ ]:
#7.3